---
title: "Ejercicio de Cálculo del IPC"
author: "Elvis Casco"
crossref:
  fig-title: Gráfico     # (default is "Figure")
  tbl-title: Tabla     # (default is "Table")
  fig-prefix: Gráfico   # (default is "Figure")
  tbl-prefix: Tabla    # (default is "Table")
format:
  # html:
  #   toc: true
  #   code-fold: true
  docx: 
    toc: true
    reference-doc: custom-reference.docx
    css: styles.css
    documentclass: report
    # classoption: landscape
    geometry: "top=1in, bottom=1in, left=0.5in, right=0.1in"
    # pandoc-args: ["--variable", "geometry:margin=0.5in"]
    echo: false
    warnings: false
    keep-ipynb: true
jupyter: python3
---

In [1]:
# %pip install --upgrade polars

# import duckdb
# import glob
# import matplotlib.pyplot as plt
import numpy as np
# import os
import polars as pl
# import pandas as pd
# import pyarrow
# import reader
# import time

from great_tables import GT, exibble, style, loc, google_font
# from math import ceil
# from multiprocessing import Pool
# from typing import Dict, Any
# from typing import Optional

wd = "C:/Directorio_Trabajo/2024/IPC_Calc/"

In [2]:
# DataFrame de ponderaciones por agrupación y región desde hoja de Excel
def get_ponds_from_xlsx():
    df = pl.read_excel(
        source = wd + "Categorias.xlsx",
        sheet_name = "Regiones",
        infer_schema_length=None,
        schema_overrides={
            "División": pl.String,
            "Grupo": pl.String,
            "Clase": pl.String,
            "SubClase": pl.String,
            "Categoría": pl.String,
            "Producto": pl.String,
            },
    )[:,1:]
    df = df.with_columns(
        pl.col("Código").str.slice(0, 2).alias("División"),
        pl.col("Código").str.slice(0, 3).alias("Grupo"),
        pl.col("Código").str.slice(0, 4).alias("Clase"),
        pl.col("Código").str.slice(0, 5).alias("SubClase"),
        pl.col("Código").str.slice(0, 6).alias("Categoría"),
        pl.col("Código").str.slice(0, 8).alias("Producto"),
        )
    return df

# DataFrame de índices desde hoja de Excel
def get_df_from_xlsx(region):
    df = pl.read_excel(
        source = wd + "Ejercicio calculo IPC - Investigación.xlsx",
        sheet_name = region
    )[:,2:]
    return df

# Solo los datos que contienen Precio, Unidad de Medida o Contenido; depura establecimientos
def get_valid_columns_in_df(df):
    nombres_t = df.columns
    df[1,0]="Precio"
    df[1,1]="Unidad de Medida"
    df[1,2]="Contenido"

    check_list=["Precio","Unidad de Medida","Contenido"] 
    row_values = df.row(1)
    conditions_met = [value in check_list for value in row_values]
    df = df[conditions_met]
    return df

# Repetir los valores de los establecimientos en la primera fila
def repeat_names_in_row(row):
    row = list(row)
    for i in range(1, len(row)):
        if i % 3 != 0:
            row[i] = row[i - (i % 3)]
    return row

def replace_names_in_df(df,row_index,modified_row):
    row_values = df.row(row_index)
    df = df.with_columns([
        pl.when(
            pl.arange(0, df.height) == row_index
            ).then(
                pl.lit(modified_row[i])
            ).otherwise(pl.col(col)
            ).alias(col)
        for i, col in enumerate(df.columns)
    ])
    return df

def modify_df_names(df):
    modified_row = repeat_names_in_row(df.row(0))
    df = replace_names_in_df(df,0,modified_row)
    return df

def replace_unnamed(row):
    for i in range(1, len(row)):
        if r"UNNAMED" in row[i]:
            row[i] = row[i - 1]
    return row

def modify_df_establecimientos(df):
    modified_row = df.columns
    df = replace_names_in_df(df,2,modified_row)
    row_index = 2
    row_values = df.row(row_index)
    modified_row = replace_unnamed(list(row_values))
    df = replace_names_in_df(df,2,modified_row)
    return df

def obtener_dataframe(region):
    df = get_df_from_xlsx(region)
    df = get_valid_columns_in_df(df)
    df = modify_df_names(df)
    df = modify_df_establecimientos(df)
    df[0,1] = "."
    a_list = ["Unidad de Medida","Codigo",'.']
    df = df.filter(
        pl.col('Codigo').str.contains_any(a_list))
    indices_to_select = [1] + list(range(3, len(df.columns)))
    columns_to_select = [df.columns[i] for i in indices_to_select]
    df = df.select(columns_to_select)
    df = df.transpose(include_header=False)
    new_column_names_row = df.row(0)
    new_column_names = list(new_column_names_row)
    rename_dict = {
        old: new for old, new in zip(df.columns, new_column_names)}
    df = df.rename(rename_dict)
    df = df[1:,:]
    return df

def obtener_precio_t_1(region):
    df = obtener_dataframe(region)
    df_Precio_t_1 = df.filter(
        pl.col('Codigo').str.contains('t-1'))
    df_Precio_t_1 = df_Precio_t_1.filter(
        pl.col('Unidad de Medida').str.contains('Precio'))
    df_Precio_t_1.write_excel(
        workbook = wd + region + "/" + region + "_Precio_t_1.xlsx")
    word = ' Precio'
    pattern = f'{word}.*'
    df_Precio_t_1 = df_Precio_t_1.with_columns(
        pl.col('Codigo').str.replace(pattern,""))
    word = ' Precio'
    pattern = f'{word}.*'
    df_Precio_t_1 = df_Precio_t_1.with_columns(
        pl.col('Codigo').str.replace(pattern,""))
    return df_Precio_t_1

def obtener_contenido_t_1(region):
    df = obtener_dataframe(region)
    df_Contenido_t_1 = df.filter(
        pl.col('Codigo').str.contains('t-1'))
    df_Contenido_t_1 = df_Contenido_t_1.filter(
        pl.col('Unidad de Medida').str.contains('Contenido'))
    df_Contenido_t_1.write_excel(
        workbook = wd + region + "/" + region + "_Contenido_t_1.xlsx")
    word = ' Precio'
    pattern = f'{word}.*'
    df_Contenido_t_1 = df_Contenido_t_1.with_columns(
        pl.col('Codigo').str.replace(pattern,""))
    word = ' Precio'
    pattern = f'{word}.*'
    df_Contenido_t_1 = df_Contenido_t_1.with_columns(
        pl.col('Codigo').str.replace(pattern,""))
    return df_Contenido_t_1

def obtener_precio_t(region):
    df = obtener_dataframe(region)
    df_Precio_t = df.filter(
        ~pl.col('Codigo').str.contains('t-1'))
    df_Precio_t = df_Precio_t.filter(
        pl.col('Unidad de Medida').str.contains('Precio'))
    df_Precio_t.write_excel(workbook = wd + region + "/" + region + "_Precio_t.xlsx")
    word = ' Precio'
    pattern = f'{word}.*'
    df_Precio_t = df_Precio_t.with_columns(
        pl.col('Codigo').str.replace(pattern,""))
    return df_Precio_t

def obtener_contenido_t(region):
    df = obtener_dataframe(region)
    df_Contenido_t = df.filter(
        ~pl.col('Codigo').str.contains('t-1'))
    df_Contenido_t = df_Contenido_t.filter(
        pl.col('Unidad de Medida').str.contains('Contenido'))
    df_Contenido_t.write_excel(workbook = wd + region + "/" + region + "_Contenido_t.xlsx")
    word = ' Precio'
    pattern = f'{word}.*'
    df_Contenido_t = df_Contenido_t.with_columns(
        pl.col('Codigo').str.replace(pattern,""))
    return df_Contenido_t

In [3]:
def obtener_indices(wd,region):
    # Precios en t
    p_t = obtener_precio_t(region)
    my_columns = p_t.columns
    cols_to_process = my_columns[3:p_t.shape[1]]
    p_t[cols_to_process] = p_t[cols_to_process].cast(pl.Float64, strict=False)
    # Contenidos en t
    c_t = obtener_contenido_t(region)
    my_columns = c_t.columns
    cols_to_process = my_columns[3:c_t.shape[1]]
    c_t[cols_to_process] = c_t[cols_to_process].cast(pl.Float64, strict=False)
    # Precios Unitarios en t
    pu_t = p_t.clone()
    pu_t[my_columns[3:pu_t.shape[1]]] = (p_t[my_columns[3:c_t.shape[1]]] / c_t[my_columns[3:c_t.shape[1]]])
    pu_t = pu_t.with_columns(pl.lit(region).alias("Región"))
    pu_t.write_excel(
        workbook = wd + region + "/" + region + "_Precio_Unitario_t.xlsx")

    # Precios en t-1
    p_t_1 = obtener_precio_t_1(region)
    my_columns = p_t_1.columns
    cols_to_process = my_columns[3:p_t_1.shape[1]]
    p_t_1[cols_to_process] = p_t_1[cols_to_process].cast(pl.Float64, strict=False)
    # Contenidos en t-1
    c_t_1 = obtener_contenido_t_1(region)
    my_columns = c_t_1.columns
    cols_to_process = my_columns[3:c_t_1.shape[1]]
    c_t_1[cols_to_process] = c_t_1[cols_to_process].cast(pl.Float64, strict=False)
    # Precios Unitarios en t-1
    pu_t_1 = p_t.clone()
    pu_t_1[my_columns[3:pu_t_1.shape[1]]] = (p_t_1[my_columns[3:c_t.shape[1]]] / c_t_1[my_columns[3:c_t.shape[1]]])
    pu_t_1 = pu_t_1.with_columns(pl.lit(region).alias("Región"))
    pu_t_1.write_excel(
        workbook = wd + region + "/" + region + "_Precio_Unitario_t_1.xlsx")

    # Índice de Precios por Establecimiento y Variedad
    i_t = p_t.clone()
    i_t[my_columns[3:i_t.shape[1]]] = (p_t[my_columns[3:c_t.shape[1]]] / c_t[my_columns[3:c_t.shape[1]]]) / (p_t_1[my_columns[3:c_t_1.shape[1]]] / c_t_1[my_columns[3:c_t_1.shape[1]]])
    i_t = i_t.with_columns(pl.lit(region).alias("Región"))
    i_t.write_excel(
        workbook = wd + region + "/" + region + "_Establecimiento.xlsx")

    # Índice de Precios por Variedad: Media Geométrica
    res_variedad = pl.DataFrame([
        pl.Series("Variedad", my_columns[3:i_t.shape[1]], dtype=pl.String)])
    res_variedad = res_variedad.with_columns(pl.col("Variedad").str.slice(0, 8).alias("Producto"))
    res_variedad = res_variedad.with_columns(
        Indice = 0.0)
    for row in range(res_variedad.shape[0]):
        res_variedad[row,2] = geometric_mean(
            i_t[res_variedad["Variedad"][row]].drop_nans() * 100)
    res_variedad = res_variedad.with_columns(pl.lit(region).alias("Región"))
    res_variedad.write_excel(
        workbook = wd + region + "/" + region + "_Variedad.xlsx")

    # Ponderación por Región
    Pond_Region = pl.read_excel(
        source = wd + "Categorias.xlsx",
        sheet_name = "Zonas",
        infer_schema_length=None)
    Pond_Region = Pond_Region[:,0:2]
    Pond_Region

    # Índice de Precios por Producto: Media Geométrica
    pond_region = Pond_Region.filter(pl.col("Región") == region)[0,0]
    res_producto = (
        res_variedad.group_by("Producto", maintain_order=True)
        .agg(
            pl.map_groups(
                exprs=["Indice"],
                function=geometric_mean)
        ))
    ponderaciones_producto_region = get_ponds_from_xlsx()
    res_producto = res_producto.join(
        ponderaciones_producto_region, 
        on="Producto")
    res_producto = res_producto.with_columns(
        (pl.col("Indice") * pl.col(region) / 100).alias("Indice_Pond_Producto_Región"))
    res_producto = res_producto.with_columns(
        (pl.col(region)).alias("Pond_Producto_Region"))
    res_producto = res_producto.with_columns(pl.lit(region).alias("Región"))
    res_producto = res_producto.with_columns(pl.lit(pond_region).alias("Pond_Región"))
    res_producto = res_producto.with_columns(
        (pl.col("Pond_Producto_Region") * pl.col("Pond_Región")).alias("Pond_IPC"))
    res_producto = res_producto.with_columns(
        (pl.col("Indice") * pl.col("Pond_IPC") / 100).alias("Indice_Pond_Producto_IPC"))
    res_producto = res_producto[
        "Producto","CCIF","División","Grupo","Clase","SubClase","Categoría","Región","Pond_Región","Pond_Producto_Region","Pond_IPC","Indice","Indice_Pond_Producto_Región","Indice_Pond_Producto_IPC"]
    res_producto.write_excel(
        workbook = wd + region + "/" + region + "_Producto.xlsx")
    res_producto.write_excel(
        workbook = wd + "Producto/" + region + "_Producto.xlsx")

    ## Índice de Precios por Agrupaciones: Media Ponderada
    agrupaciones = ["Categoría","SubClase","Clase","Grupo","División"]
    grupo = "Categoría"
    res_categoria = weighted_index_group_region(res_producto,grupo)
    res_categoria = res_categoria.with_columns(pl.lit(region).alias("Región"))
    res_categoria.write_excel(
        workbook = wd + grupo + "/" + region + "_" + grupo + ".xlsx")
    grupo = "SubClase"
    res_subclase = weighted_index_group_region(res_producto,grupo)
    res_subclase = res_subclase.with_columns(pl.lit(region).alias("Región"))
    res_subclase.write_excel(
        workbook = wd + grupo + "/" + region + "_" + grupo + ".xlsx")
    grupo = "Clase"
    res_clase = weighted_index_group_region(res_producto,grupo)
    res_clase = res_subclase.with_columns(pl.lit(region).alias("Región"))
    res_clase.write_excel(
        workbook = wd + grupo + "/" + region + "_" + grupo + ".xlsx")
    grupo = "Grupo"
    res_grupo = weighted_index_group_region(res_producto,grupo)
    res_grupo = res_grupo.with_columns(pl.lit(region).alias("Región"))
    res_grupo.write_excel(
        workbook = wd + grupo + "/" + region + "_" + grupo + ".xlsx")
    grupo = "División"
    res_division = weighted_index_group_region(res_producto,grupo)
    res_division = res_division.with_columns(pl.lit(region).alias("Región"))
    res_division.write_excel(
        workbook = wd + grupo + "/" + region + "_" + grupo + ".xlsx")

    # Resultados en DataFrames
    return p_t,c_t,p_t_1,c_t_1,pu_t,pu_t_1,i_t,res_variedad,res_producto,res_categoria,res_subclase,res_clase,res_grupo,res_division

# Function to calculate geometric mean
def geometric_mean(series):
    return np.exp(np.log(series).mean())

def weighted_index_group_region(df,grupo):
    result = df.group_by(grupo).agg(
        [
            (pl.col("Indice") * pl.col("Pond_Producto_Region") / 100).sum(
            ).alias("weighted_sum"),
            pl.col("Pond_Producto_Region").sum(
            ).alias("Peso_" + grupo)
        ]).with_columns([
            (pl.col("weighted_sum") / pl.col("Peso_" + grupo) * 100
            ).alias("Índice_" + grupo)
        ]).select([grupo, "Peso_" + grupo, "Índice_" + grupo
        ]).sort(grupo)
    result = result.with_columns(pl.lit(region).alias("Región"))
    result = result.with_columns(
        (pl.col("Índice_" + grupo) * pl.col("Peso_" + grupo) / 100).alias("Indice_Pond_Region"),
    )
    return result

# Function to format numerical values 
def format_number(x): 
    if isinstance(x, (int, float)): 
        return f"{x:.8f}" 
        return x

def weighted_index(group):
    return (group['Indice'] * group['Pond_IPC']).sum() / group['Pond_IPC'].sum()

def obtener_weighted_index_by_group(df,group):
    weights_by_ipc = df.group_by(group).agg(pl.col('Pond_IPC').sum().alias('w_{IPC}'))
    weighted_indices_by_group = df.group_by(group).agg(
        (pl.col('Indice') * pl.col('Pond_IPC')).sum() / pl.col('Pond_IPC').sum())
    result = weights_by_ipc.join(weighted_indices_by_group, on=group)
    result = result.with_columns(
        (pl.col('w_{IPC}') / 100).alias('w_{IPC}'))
    result = result.with_columns(
        (pl.col('w_{IPC}') / result['w_{IPC}'].sum()).alias('w_{b}'))  
    result = result.rename({"Indice": "I^{0:t}"})
    return result

def obtener_numerical_columns(df):
    numerical_columns = [col for col in df.columns if df[col].dtype in [pl.Float64, pl.Int64]]
    return numerical_columns

In [4]:
regions = ["MDC","RUC","MSPS","RUN","ULA","UOri","UOcc","US"]
for i, region in enumerate(regions):
    res = obtener_indices(wd,region)
# {{< pagebreak >}}

**Nota:** Para facilitar el entendimiento, primero se expone una breve explicación de conceptos y cálculos, tomada literalmente del Manual, partiendo de los elementos más pequeños: variedades por establecimiento. En dicho manual, a estos se les nombra como **agregados elementales**. Posteriormente, se presentan los resultados tomando en cuenta datos simulados, remitidos por GIE.

# Índices, agregados elementales

## Construcción de agregados elementales

Los agregados elementales son grupos de bienes y servicios relativamente homogéneos, que pueden abarcar todo el país o solo regiones individuales. Asimismo, pueden establecerse distintos agregados elementales para distintos tipos de puntos de venta.

Para su escogencia se tienen en cuenta los siguientes elementos:

- Los agregados elementales deberían componerse de grupos de bienes o servicios tan parecidos entre sí como sea posible y, preferentemente, homogéneos.
- Deberían estar compuestos de artículos de los cuales se esperan variaciones de precios parecidas, a efectos de minimizar la dispersión de las variaciones de precios dentro del agregado. 
- Los agregados elementales deberían ser apropiados para servir como estratos para propósito de muestreo en función del régimen de muestreo que se establezca para la recopilación de datos.

Utilizando una clasificación de los gastos del consumidor como la Clasificación del Consumo Individual por Finalidades (CCIF), todo el conjunto de bienes y servicios de consumo que abarca el índice de precios al consumidor (IPC) nivel general puede dividirse en **grupos**, por ejemplo "comestibles y bebidas no alcohólicas". Cada grupo se divide a su vez en **clases**, por ejemplo, "comestibles". A los fines del IPC, cada clase puede dividirse a su vez en **subclases** más homogéneas, como "arroz". Las subclases equivalen a los capítulos del Programa de Comparación Internacional, que calcula las paridades de poder adquisitivo (PPA) entre países. Finalmente, la subclase puede descomponerse aún más para obtener agregados elementales por regiones o puntos de venta.

Dentro de cada agregado elemental se seleccionan uno o más artículos para representar todos los artículos pertenecientes a él. Por ejemplo, el agregado elemental compuesto por arroz en venta en supermercados del norte del país cubre todos los tipos de arroz, de los cuales se seleccionan como artículos representativos el arroz blanco precocido y el integral con más del 50% de granos partidos. Desde luego, en la práctica puede seleccionarse una mayor cantidad de artículos representativos. Finalmente, por cada uno puede seleccionarse una cantidad de productos específicos para la recopilación de precios, por ejemplo determinadas marcas de arroz precocido. Nuevamente, la cantidad de productos seleccionados para la muestra dependerá de la naturaleza del
producto representativo.

A continuación se analizan los métodos utilizados para calcular índices elementales a partir de observaciones sobre precios individuales. Todos los índices de nivel superior por encima del nivel agregado elemental se obtienen a partir de los índices de precios elementales utilizando como ponderaciones los agregados elementales de gastos. 

La estructura de agregación es consistente, de manera que la ponderación de cada nivel por encima del agregado elemental siempre es igual a la suma de sus componentes. El índice de precios en cada nivel superior de agregación se calcula sobre la base de las ponderaciones y los índices de precios de sus componentes, es decir, los índices de nivel inferior o elementales.

Los índices de precios elementales individuales pueden no ser lo suficientemente confiables para publicarlos por separado, aunque sirven para construir todos los índices de nivel superior.

En la mayoría de los casos, los índices de precios de los agregados elementales se calculan sin utilizar ponderaciones de gasto explícitas. Sin embargo, en la medida de lo posible, deberían utilizarse ponderaciones que reflejen la importancia relativa de los artículos incluidos en la muestra, aun si las ponderaciones son solo aproximadas. 

A menudo, el agregado elemental es sencillamente el nivel más bajo respecto del cual se dispone de información confiable. En este caso, el índice elemental debe calcularse como el promedio no ponderado de los precios que lo componen. 

## Construcción de índices de precios elementales

Un índice de precios elemental es un índice de precios de un agregado elemental. En el ejemplo se supone que para el agregado elemental se recopilan los precios de cuatro artículos. La calidad de cada artículo permanece constante a lo largo del tiempo a efectos de asegurar que las variaciones de un mes a otro representen una comparación entre semejantes. 

Inicialmente se supone que los precios de los cuatro artículos se recopilan todos los meses, con lo cual se dispone de un conjunto completo de precios. No desaparece ningún artículo ni falta ningún precio, y tampoco hay artículos de reemplazo. Se trata de un supuesto bastante fuerte, pues muchos de los problemas que surgen en la práctica se deben a rupturas en la continuidad de la serie de precios de los artículos individuales, por cualquier motivo. 

Tres fórmulas muy utilizadas por las oficinas de estadística para calcular los índices de precios elementales. Sin embargo, cabe tener en cuenta que no constituyen las únicas posibilidades y más adelante se consideran algunas fórmulas alternativas.

1. **Índice de Carli** para $i = 1,..., n$ artículos. Se define como la media aritmética simple, o no ponderada, de los cocientes relativos de precios, o cocientes de precios, de los dos períodos, 0 y t, que se comparan:

$I^{0:t}_C=\frac{1}{n}\sum (\frac{p_i^t}{p_i^0})$

2. **Índice de Dutot**, que se define como el cociente de las medias aritméticas no ponderadas de los precios:

$I^{0:t}_D=\frac{\frac{1}{n}\sum p_i^t}{\frac{1}{n}\sum p_i^0}$

3. **Índice de Jevons**, que se define como la media geométrica no ponderada de los relativos de precios o cocientes, que es idéntica al cociente de las medias geométricas simples de los precios:

$I^{0:t}_J=\prod (\frac{p_i^t}{p_i^0})^{\frac{1}{n}} = \frac{\prod (p_i^t)^{\frac{1}{n}}}{\prod (p_i^0)^{\frac{1}{n}}}$

## Ejemplo de procedimiento para índices de precios elementales: Arroz Clasificado, Región MDC


Para Honduras, a partir del código CCIF, se tienen productos por CCIF agrupados por:

- Producto: CCIF a 8 dígitos;
- Categoría: CCIF a 6 dígitos;
- Subclase: CCIF a 5 dígitos;
- Clase: CCIF a 4 dígitos;
- Grupo: CCIF a 3 dígitos; y
- División: CCIF a 2 dígitos.

Las regiones a considerarse son:

- MDC = Metropolitana Distrito Central
- RUC = Resto Urbano Central
- MSPS =  Metropolitana San Pedro Sula
- RUN = Resto Urbano Norte
- ULA = Urbana Litoral Atlántico
- UOri = Urbana Oriental
- UOcc = Urbana Occidental
- US = Urbana Sur

En la tabla se muestran los datos de los primeros 5 productos con sus respectivas ponderaciones para la región MDC.


In [5]:
regions = ["MDC","RUC","MSPS","RUN","ULA","UOri","UOcc","US"]
ponderaciones_producto_region = get_ponds_from_xlsx()
(
    GT(ponderaciones_producto_region[0:5,0:9])
    .fmt_number(columns="MDC", decimals=4)
    .sub_missing()
    .tab_style(
        style=style.text(size="14px"),
        locations=loc.body(columns=[
            "Código","CCIF","División","Grupo","Clase","SubClase","Categoría","Producto"]))
        # locations=loc.body(columns=ponderaciones_producto_region.columns))
)

Código,CCIF,División,Grupo,Clase,SubClase,Categoría,Producto,MDC
01111201,Arroz clasificado,01,011,0111,01111,011112,01111201,0.5700
01111601,Maíz en grano o desgranado,01,011,0111,01111,011116,01111601,0.4200
01112101,Harina de trigo,01,011,0111,01112,011121,01112101,0.0400
01112601,Harina de maíz,01,011,0111,01112,011126,01112601,0.0500
01113101,Bollito de yema pan dulce,01,011,0111,01113,011131,01113101,0.0100


Un ejemplo de composición de agregados elementales para Honduras, correspondiente al arroz clasificado (CCIF= 01111201) en MDC es:


In [6]:
region = "MDC"
p_t,c_t,p_t_1,c_t_1,pu_t,pu_t_1,i_t,res_variedad,res_producto,res_categoria,res_subclase,res_clase,res_grupo,res_division = obtener_indices(wd,region)

In [7]:
GT(p_t[:,0:6].filter(
   ~pl.all_horizontal(pl.col(pl.Float64).is_nan())
))

.,Unidad de Medida,Codigo,01111201.01,01111201.02,01111201.03
Supermercado 1,Precio,Supermercado,55.1096,NaN,20.0345
Supermercado 2,Precio,Supermercado,58.40515408,NaN,23.61065825
Mercadito 1,Precio,Mercadito,NaN,12.2705,NaN
Mercadito 2,Precio,Mercadito,NaN,12.8205,NaN


Este producto tiene tres subclasificaciones o variedades:

- 01111201.01	Arroz clasificado;
- 01111201.02	Arroz corriente; y
- 01111201.03	Arroz precocido.

Además, se consultan precios en dos tipos de establecimiento:

- Supermercado (1 y 2); y
- Mercadito (1 y 2)

El componente más pequeño que se definió es la variedad, misma que contiene datos de **precios** y **contenido** (conversión a unidad de medida); el **precio unitario** se obtiene mediante la división de los elementos de estas tablas:

### 1. Precios en t, por establecimiento y producto:


In [8]:
GT(p_t[:,0:6].filter(
   ~pl.all_horizontal(pl.col(pl.Float64).is_nan())
))

.,Unidad de Medida,Codigo,01111201.01,01111201.02,01111201.03
Supermercado 1,Precio,Supermercado,55.1096,NaN,20.0345
Supermercado 2,Precio,Supermercado,58.40515408,NaN,23.61065825
Mercadito 1,Precio,Mercadito,NaN,12.2705,NaN
Mercadito 2,Precio,Mercadito,NaN,12.8205,NaN


### 2. Contenido en t, por establecimiento y producto:


In [9]:
GT(c_t[:,0:6].filter(
   ~pl.all_horizontal(pl.col(pl.Float64).is_nan())
))

.,Unidad de Medida,Codigo,01111201.01,01111201.02,01111201.03
Supermercado 1,Contenido,Supermercado,1.75,NaN,350.0
Supermercado 2,Contenido,Supermercado,1.75,NaN,350.0
Mercadito 1,Contenido,Mercadito,NaN,1.0,NaN
Mercadito 2,Contenido,Mercadito,NaN,1.0,NaN


### 3. Precios unitarios (Precios / Contenido) en t, por establecimiento y producto:


In [10]:
GT(pu_t[:,0:6].filter(
   ~pl.all_horizontal(pl.col(pl.Float64).is_nan())
))

.,Unidad de Medida,Codigo,01111201.01,01111201.02,01111201.03
Supermercado 1,Precio,Supermercado,31.4912,NaN,0.057241428571428575
Supermercado 2,Precio,Supermercado,33.374373760000005,NaN,0.06745902357142858
Mercadito 1,Precio,Mercadito,NaN,12.2705,NaN
Mercadito 2,Precio,Mercadito,NaN,12.8205,NaN


### 4. Precios en t-1, por establecimiento y producto:


In [11]:
GT(p_t_1[:,0:6].filter(
   ~pl.all_horizontal(pl.col(pl.Float64).is_nan())
))

.,Unidad de Medida,Codigo,01111201.01,01111201.02,01111201.03
Supermercado 1,Precio,Supermercado,52.0,NaN,17.0
Supermercado 2,Precio,Supermercado,59.8,NaN,17.85
Mercadito 1,Precio,Mercadito,NaN,11.0,NaN
Mercadito 2,Precio,Mercadito,NaN,11.55,NaN


### 5. Contenido en t-1, por establecimiento y producto:


In [12]:
GT(c_t_1[:,0:6].filter(
   ~pl.all_horizontal(pl.col(pl.Float64).is_nan())
))

.,Unidad de Medida,Codigo,01111201.01,01111201.02,01111201.03
Supermercado 1,Contenido,Supermercado,1.75,NaN,350.0
Supermercado 2,Contenido,Supermercado,1.75,NaN,350.0
Mercadito 1,Contenido,Mercadito,NaN,1.0,NaN
Mercadito 2,Contenido,Mercadito,NaN,1.0,NaN


### 6. Precios unitarios (Precios / Contenido) en t-1, por establecimiento y producto:


In [13]:
GT(pu_t_1[:,0:6].filter(
   ~pl.all_horizontal(pl.col(pl.Float64).is_nan())
))

.,Unidad de Medida,Codigo,01111201.01,01111201.02,01111201.03
Supermercado 1,Precio,Supermercado,29.714285714285715,NaN,0.04857142857142857
Supermercado 2,Precio,Supermercado,34.17142857142857,NaN,0.051000000000000004
Mercadito 1,Precio,Mercadito,NaN,11.0,NaN
Mercadito 2,Precio,Mercadito,NaN,11.55,NaN


### 7. Índices individuales en t, por establecimiento y producto  (Índice de Carli): 

Precio unitario en t / Precio Unitario en t-1

$I^{0:t}_C=\frac{1}{n}\sum (\frac{p_i^t}{p_i^0})$


In [14]:
GT(i_t[:,0:6].filter(
   ~pl.all_horizontal(pl.col(pl.Float64).is_nan())
))

.,Unidad de Medida,Codigo,01111201.01,01111201.02,01111201.03
Supermercado 1,Precio,Supermercado,1.0597999999999999,NaN,1.1785
Supermercado 2,Precio,Supermercado,0.9766748173913045,NaN,1.3227259523809525
Mercadito 1,Precio,Mercadito,NaN,1.1155,NaN
Mercadito 2,Precio,Mercadito,NaN,1.1099999999999999,NaN


### 8. Índice en t, por variedad (Índice de Jevons):

Media geométrica de los Índices individuales en t, por establecimiento y producto (por cada columna, en la tabla anterior).

$I^{0:t}_{Jv}=\prod (\frac{p_i^t}{p_i^0})^{\frac{1}{n}} = \frac{\prod (p_i^t)^{\frac{1}{n}}}{\prod (p_i^0)^{\frac{1}{n}}}$


In [15]:
GT(res_variedad[0:3,:].filter(
   ~pl.all_horizontal(pl.col(pl.Float64).is_nan())
))

Variedad,Producto,Indice,Región
01111201.01,01111201,101.73888005434813,MDC
01111201.02,01111201,111.27466018820274,MDC
01111201.03,01111201,124.85321521214229,MDC


### 9. Índice en t, por producto:

Media geométrica de los Índices individuales en t, por variedad.

$I^{0:t}_{Jp}=\prod (I_{Jv1}^{0:t},I_{Jv2}^{0:t},...,I_{Jvn}^{0:t})^\frac{1}{n}$

$I^{0:t}_{Jp}= \exp \bigg(\frac{{\ln (I_{Jv1}^{0:t}) + \ln (I_{Jv2}^{0:t}) +...+\ln (I_{Jvn}^{0:t})}}{n}\bigg)$


In [16]:
GT(res_producto[0:1,0:12].filter(
   ~pl.all_horizontal(pl.col(pl.Float64).is_nan())
)).fmt_number(columns="Indice", decimals=4)

Producto,CCIF,División,Grupo,Clase,SubClase,Categoría,Región,Pond_Región,Pond_Producto_Region,Pond_IPC,Indice
01111201,Arroz clasificado,01,011,0111,01111,011112,MDC,0.34,0.57,0.1938,112.2262


# Cálculo de índices de nivel superior

Las oficinas de estadística deben apuntar a algún índice objetivo o meta. Para ello  deben considerar qué tipo de índice elaborarían en una situación hipotética ideal en que contaran con toda la información necesaria sobre los precios y las cantidades en los dos períodos comparados. 

Si el IPC tiene por finalidad servir de índice del costo de vida, un índice superlativo de tipo Fisher, Walsh o Törnqvist-Theil serviría como objetivo teórico, pues se espera que un índice superlativo se aproxime al índice del costo de vida subyacente.

Muchos países intentan calcular un índice del costo de vida y prefieren el concepto de índice basado en una canasta. Un índice basado en una canasta mide la variación en el valor total de una determinada canasta de bienes y servicios entre dos períodos. En este manual, esta categoría general de índice se define como un índice de Lowe.

En teoría puede elegirse cualquier índice objetivo. En la práctica, es probable que se prefiera un índice de Laspeyres o cualquier otro índice superlativo.

## Índices de precios al consumidor como promedios ponderados de índices elementales

Un índice de nivel superior es un índice de determinado agregado de gasto por encima del nivel de un agregado elemental como, por ejemplo, el IPC nivel general. Los datos que se utilizan para calcular índices de nivel superior son:

– Los índices de precios elementales.
– Las ponderaciones obtenidas de los valores de agregados elementales en varios años anteriores.

Los índices de nivel superior se calculan simplemente como promedios aritméticos ponderados de los índices de precios elementales. Esta categoría general de índice se define en este manual como índice de Young.

La segunda etapa de la elaboración del IPC no comprende precios ni cantidades individuales. Por el contrario, un índice de nivel superior es un índice de Young en el cual los índices de precios elementales se promedian utilizando un conjunto de ponderaciones predeterminadas.

La fórmula puede plantearse de la siguiente manera:

$I^{0:t}=\sum w^b_i I^{0:t}_i$, $\sum w^b_i=1$

- $I^{0:t}$ denota el IPC nivel general, o cualquier índice de nivel superior, entre el período 0 y $t$;
- $w^b_i$ es la ponderación asignada a cada índice de precios elemental;
- $I^{0:t}_i$ es el índice de precios elemental correspondiente.

Los índices elementales se identifican con el subíndice $i$, mientras que el índice de nivel superior no lleva subíndice. Como ya se señaló, un índice superior es cualquier índice, incluido el IPC nivel general, por encima del nivel del agregado elemental. Las ponderaciones se obtienen de los gastos en el período $b$, que en la práctica debe ser anterior al período 0, el período de referencia de los precios.

## Ejemplo de procedimiento para índices de nivel superior: por productos, regiones y elementos de la CCIF

En el el ejercicio propuesto, la ponderación de las regiones es la siguiente:


In [17]:
IPC = pl.read_excel(
    source = wd + "Producto/" + regions[0] + "_Producto.xlsx",
    infer_schema_length=None)
for i, region in enumerate(regions[1:len(regions)]):
    IPC = pl.concat([
        IPC, 
        pl.read_excel(
            source = wd + "Producto/" + region + "_Producto.xlsx",
            infer_schema_length=None)], 
        how="vertical_relaxed")
IPC.write_excel(
    workbook = wd + "IPC.xlsx")

df = pl.read_excel(
        source = wd + "Categorias.xlsx",
        sheet_name = "Zonas"
    )[:,:3]
df
GT(df).fmt_number(columns="Ponderación", decimals=4)

Ponderación,Región,Nombre_Región
0.3400,MDC,1 Metropolitana Distrito Central
0.0500,RUC,2 Resto Urbano Central
0.3100,MSPS,3 Metropolitana San Pedro Sula
0.1000,RUN,4 Resto Urbano Norte
0.0700,ULA,5 Urbano Litoral Atlántico
0.0300,UOri,6 Urbana Oriental
0.0600,UOcc,7 Urbana Occidental
0.0400,US,8 Urbana Sur


El cálculo del IPC por región y general implica que se debe tomar en cuenta tanto la ponderación del producto por región y la ponderación de la región en el IPC. 

Recordando que por cada producto se tiene la siguiente forma de agrupación (columnas):


In [18]:
GT(res_producto[0:1,0:9].filter(
   ~pl.all_horizontal(pl.col(pl.Float64).is_nan())
))

Producto,CCIF,División,Grupo,Clase,SubClase,Categoría,Región,Pond_Región
01111201,Arroz clasificado,01,011,0111,01111,011112,MDC,0.34


Para todos los productos del IPC, las **agrupaciones** tomando en cuenta el CCIF (partiendo de lo específico a lo general) son:

1. Producto;
2. Categoría;
3. SubClase;
4. Clase:
5. Grupo; y
6. División.

Tomando en cuenta la fórmula explicada previamente:

$I^{0:t}=\sum w^b_i I^{0:t}_i$, $\sum w^b_i=1$

- $I^{0:t}$ denota el IPC nivel general, o cualquier índice de nivel superior, entre el período 0 y $t$;
- $w^b_i$ es la ponderación asignada a cada índice de precios elemental; y
- $I^{0:t}_i$ es el índice de precios elemental correspondiente.

Se pueden generar, a partir de los índices por producto y región y las ponderaciones por producto y región, combinaciones de agrupaciones y regiones para llegar al IPC general

- IPC por productos y regiones
- IPC por productos;
- IPC por agrupaciones;
- IPC por regiones y agrupaciones;
- IPC general.

### IPC por productos y regiones

Si seguimos con el ejemplo del arroz clasificado, los resultados por región son:


In [19]:
# Crear dataframe para IPC
columns = ['Agrupación',"IPC"]
# IPC_General = pl.DataFrame({col: [] for col in columns})

grupos = ["Producto","CCIF","Región"]
producto = "01111201"
result = obtener_weighted_index_by_group(IPC,grupos).sort(grupos)
result.write_excel(
    workbook = wd + "IPC_Productos_Regiones.xlsx")
ipc_calc = (result["I^{0:t}"]*result["w_{b}"]).sum()
# print("IPC = " + str((ipc_calc)))
IPC_General = pl.DataFrame({'Agrupación': str(grupos), 'IPC': ipc_calc})

arroz = result.filter(pl.col("Producto") == producto)
GT(arroz.sort("Producto")).fmt_number(columns=obtener_numerical_columns(arroz), decimals=6)

Producto,CCIF,Región,w_{IPC},I^{0:t},w_{b}
01111201,Arroz clasificado,MDC,0.001938,112.226216,0.001938
01111201,Arroz clasificado,MSPS,0.001088,111.961215,0.001088
01111201,Arroz clasificado,RUC,0.000729,111.700533,0.000729
01111201,Arroz clasificado,RUN,0.000409,110.698498,0.000409
01111201,Arroz clasificado,ULA,0.000380,111.091757,0.000380
01111201,Arroz clasificado,UOcc,0.000205,112.387346,0.000205
01111201,Arroz clasificado,UOri,0.000139,111.292105,0.000139
01111201,Arroz clasificado,US,0.000185,113.914934,0.000185


El mismo procedimiento se aplica para el resto de los productos. En la tabla los valores se aproximan a 6 decimales, sin embargo, las operaciones dentro de los cálculos se realizan con todos los decimales disponibles.

### IPC por productos

También puede obtenerse el IPC de este y los demás productos (I^{0:t}). Por razones de espacio, solamente se detallan los pertenecientes a la clase "0111":


In [20]:
grupos = ["Producto","CCIF"]
result = obtener_weighted_index_by_group(IPC,grupos).sort(grupos)
result.write_excel(
    workbook = wd + "IPC_Producto.xlsx")
result
ipc_calc = (result["I^{0:t}"]*result["w_{b}"]).sum()
# print("IPC = " + str((ipc_calc)))
row = pl.DataFrame({'Agrupación': str(grupos), 'IPC': ipc_calc})
IPC_General = IPC_General.vstack(row)
GT(result[0:18].sort("Producto")).fmt_number(columns=obtener_numerical_columns(result), decimals=6)

Producto,CCIF,w_{IPC},I^{0:t},w_{b}
01111201,Arroz clasificado,0.005073,111.928271,0.005073
01111601,Maíz en grano o desgranado,0.008860,106.149894,0.008860
01112101,Harina de trigo,0.002546,117.420314,0.002546
01112601,Harina de maíz,0.009282,140.990288,0.009282
01113101,Bollito de yema pan dulce,0.000701,112.805845,0.000701
01113117,Pan blanco,0.003005,125.623445,0.003005
01113119,Pan molde blanco,0.002884,105.847932,0.002884
01113127,Tortillas de harina de trigo,0.002048,139.541938,0.002048
01113128,Tortillas de maíz (artesanales),0.003511,111.595330,0.003511
01113133,Rosquilla,0.007356,106.211893,0.007356


### IPC por elementos de CCIF

#### Categorías

Por razones de espacio, solamente se detallan los pertenecientes a la clase "0111":


In [21]:
grupos = ["Categoría"]
result = obtener_weighted_index_by_group(IPC,grupos).sort(grupos)
result.write_excel(
    workbook = wd + "IPC_" + grupos[0] + ".xlsx")
# print("IPC = " + str((ipc_calc)))
row = pl.DataFrame({'Agrupación': str(grupos), 'IPC': ipc_calc})
IPC_General = IPC_General.vstack(row)
GT(result[0:10].sort(grupos)).fmt_number(columns=obtener_numerical_columns(result), decimals=6)

Categoría,w_{IPC},I^{0:t},w_{b}
011112,0.005073,111.928271,0.005073
011116,0.008860,106.149894,0.008860
011121,0.002546,117.420314,0.002546
011126,0.009282,140.990288,0.009282
011131,0.019505,113.854574,0.019505
011139,0.015282,126.037753,0.015282
011140,0.011684,106.486801,0.011684
011150,0.009279,121.336811,0.009279
011190,0.000507,118.613283,0.000507
011193,0.005783,123.105295,0.005783


#### SubClases

Se detallan los pertenecientes a la clase "0111":


In [22]:
grupos = ["SubClase"]
result = obtener_weighted_index_by_group(IPC,grupos).sort(grupos)
result.write_excel(
    workbook = wd + "IPC_" + grupos[0] + ".xlsx")
# print("IPC = " + str((ipc_calc)))
row = pl.DataFrame({'Agrupación': str(grupos), 'IPC': ipc_calc})
IPC_General = IPC_General.vstack(row)
GT(result[0:6].sort(grupos)).fmt_number(columns=obtener_numerical_columns(result), decimals=6)

SubClase,w_{IPC},I^{0:t},w_{b}
01111,0.013933,108.253782,0.013933
01112,0.011828,135.916503,0.011828
01113,0.034787,119.206614,0.034787
01114,0.011684,106.486801,0.011684
01115,0.009279,121.336811,0.009279
01119,0.006291,122.743116,0.006291


#### Clases

Se detallan los pertenecientes a la división "01":


In [23]:
grupos = ["Clase"]
result = obtener_weighted_index_by_group(IPC,grupos).sort(grupos)
result.write_excel(
    workbook = wd + "IPC_" + grupos[0] + ".xlsx")
# print("IPC = " + str((ipc_calc)))
row = pl.DataFrame({'Agrupación': str(grupos), 'IPC': ipc_calc})
IPC_General = IPC_General.vstack(row)
GT(result[0:15].sort(grupos)).fmt_number(columns=obtener_numerical_columns(result), decimals=6)

Clase,w_{IPC},I^{0:t},w_{b}
0111,0.087802,118.505449,0.087802
0112,0.103385,110.087401,0.103385
0113,0.008272,116.277059,0.008272
0114,0.029221,114.620145,0.029221
0115,0.011969,127.550422,0.011969
0116,0.038942,109.949159,0.038942
0117,0.073573,122.295407,0.073573
0118,0.008196,111.149443,0.008196
0119,0.054980,111.459137,0.054980
0121,0.007947,106.036207,0.007947


#### Grupos


In [24]:
grupos = ["Grupo"]
result = obtener_weighted_index_by_group(IPC,grupos).sort(grupos)
result.write_excel(
    workbook = wd + "IPC_" + grupos[0] + ".xlsx")
# print("IPC = " + str((ipc_calc)))
row = pl.DataFrame({'Agrupación': str(grupos), 'IPC': ipc_calc})
IPC_General = IPC_General.vstack(row)
GT(result.sort(grupos)).fmt_number(columns=obtener_numerical_columns(result), decimals=6)

Grupo,w_{IPC},I^{0:t},w_{b}
011,0.416340,115.152277,0.416340
012,0.021150,119.514935,0.021150
031,0.063480,104.427832,0.063480
032,0.131817,110.834962,0.131817
051,0.030959,110.852111,0.030959
052,0.020105,113.112743,0.020105
053,0.021155,111.890142,0.021155
054,0.019466,119.156867,0.019466
055,0.009874,121.767631,0.009874
056,0.051119,128.731940,0.051119


#### Divisiones


In [25]:
grupos = ["División"]
result = obtener_weighted_index_by_group(IPC,grupos).sort(grupos)
result.write_excel(
    workbook = wd + "IPC_" + grupos[0] + ".xlsx")
# print("IPC = " + str((ipc_calc)))
row = pl.DataFrame({'Agrupación': str(grupos), 'IPC': ipc_calc})
IPC_General = IPC_General.vstack(row)
GT(result.sort(grupos)).fmt_number(columns=obtener_numerical_columns(result), decimals=6)

División,w_{IPC},I^{0:t},w_{b}
01,0.437489,115.363182,0.437489
03,0.195297,108.752362,0.195297
05,0.152677,119.044836,0.152677
06,0.214537,113.032061,0.214537


### IPC por regiones


In [26]:
grupos = ["Región"]
result = obtener_weighted_index_by_group(IPC,grupos).sort(grupos)
result.write_excel(
    workbook = wd + "IPC_Regiones.xlsx")
# print("IPC = " + str((ipc_calc)))
row = pl.DataFrame({'Agrupación': str(grupos), 'IPC': ipc_calc})
IPC_General = IPC_General.vstack(row)
GT(result).fmt_number(columns=obtener_numerical_columns(result), decimals=6)

Región,w_{IPC},I^{0:t},w_{b}
MDC,0.340000,114.153662,0.340000
MSPS,0.310000,113.969505,0.310000
RUC,0.050000,114.421439,0.050000
RUN,0.100000,113.456322,0.100000
ULA,0.070000,115.052557,0.070000
UOcc,0.060000,114.205524,0.060000
UOri,0.030000,114.316006,0.030000
US,0.040000,114.727948,0.040000


Igualmente puede aplicarse la fórmula para obtener componentes del IPC por regiones, por ejemplo divisiones por región; por razones de espacio, se muestra solamente la correspondiente a la División "01":


In [27]:
grupos = ["División","Región"]
result = obtener_weighted_index_by_group(IPC,grupos).sort(grupos)
result.write_excel(
    workbook = wd + "IPC_Regiones_Divisiones.xlsx")
# print("IPC = " + str((ipc_calc)))
row = pl.DataFrame({'Agrupación': str(grupos), 'IPC': ipc_calc})
IPC_General = IPC_General.vstack(row)
GT(result[0:8]).fmt_number(columns=obtener_numerical_columns(result), decimals=6)

División,Región,w_{IPC},I^{0:t},w_{b}
01,MDC,0.143558,115.684383,0.143558
01,MSPS,0.141175,114.452628,0.141175
01,RUC,0.021170,115.189520,0.021170
01,RUN,0.031054,115.791194,0.031054
01,ULA,0.037173,116.892579,0.037173
01,UOcc,0.031680,115.738589,0.031680
01,UOri,0.014073,116.074167,0.014073
01,US,0.017607,115.026439,0.017607


### IPC general

En todos los casos presentados, el IPC general se obtiene mediante:

$IPC^{0:t}=\sum w^b_i I^{0:t}_i$, $\sum w^b_i=1$


In [28]:
GT(IPC_General).fmt_number(columns=obtener_numerical_columns(IPC_General), decimals=8)

Agrupación,IPC
"['Producto', 'CCIF', 'Región']",114.13410428
"['Producto', 'CCIF']",114.13410428
['Categoría'],114.13410428
['SubClase'],114.13410428
['Clase'],114.13410428
['Grupo'],114.13410428
['División'],114.13410428
['Región'],114.13410428
"['División', 'Región']",114.13410428
